In [1]:
import numpy as np

In [2]:
# assume n,T,k,y,x,W_ls are given and known

In [3]:
def get_Znt(n,k,x,y,W_ls,t):
    Y_lag = np.array(y[t]).reshape(n,1)
    WY_lag = np.matmul(W_ls[t],Y_lag).reshape(n,1)
    Xnt = np.array(x[t]).reshape(n,k)
    Znt = regroup_matrix(Y_lag,WY_lag,Xnt,n,k)
    return Znt

In [4]:
def regroup_matrix(y_vec, wy_vec, x_vec, n, k):
    # regroup n*1, n*1, n*k into a matrix of n*(k+2)
    my_list = [y_vec,wy_vec,x_vec]
    reshape_list = []
    for i in range(n):
        row_vec = [*my_list[0][i],*my_list[1][i]]
        row_vec.extend(my_list[2][i])
        reshape_list.append(row_vec)
    return np.array(reshape_list).reshape(n,k+2)  

In [5]:
def get_rnt(sigma,lam,gamma, rho, beta, t):
    # first part
    delta = np.array([gamma,rho,*beta]).reshape(k+2,1)
    W = W_ls[t+1]
    S_nt = np.identity(n) - lam*W # n-by-n
    SlamY = np.matmul(lam*S_nt,y[t+1]).reshape(n,1) # n-by-1
    # second part
    Znt = get_Znt(n,k,x,y,W_ls,t) # n-by-(k+2)
    Zdelta = np.matmul(Znt,delta).reshape(n,1)
    # mesh together
    rnt = SlamY-Zdelta
    return rnt

In [6]:
# no lag
def get_c(sigma,lam,gamma, rho, beta):
    sum_ls = []
    for i in range(T):
        rnt = get_rnt(sigma,lam,gamma, rho, beta, i)
        sum_ls.append(rnt)
    c0 = (1/T)*sum(sum_ls)
    return c0       

In [7]:
def get_alpha_t(sigma,lam,gamma, rho, beta, t):
    l_n = np.ones(n).reshape(n,1)
    rnt = get_rnt(sigma,lam,gamma, rho, beta, t)
    c0 = get_c(sigma,lam,gamma, rho, beta)
    alpha_t = (1/n)*np.matmul(l_n.transpose(),rnt-c0)[0][0]
    return alpha_t

In [8]:
# test "regroup_matrix"
n = 3
k = 1
y_vec = np.array([1,2,3]).reshape(n,1)
wy_vec = np.array([2,3,4]).reshape(n,1)
x_vec = np.array([0,1,2]).reshape(n,k)
regroup_matrix(y_vec, wy_vec, x_vec, n, k)

array([[1, 2, 0],
       [2, 3, 1],
       [3, 4, 2]])

In [9]:
# test "regroup_matrix"
n = 3
k = 2
y_vec = np.array([1,2,3]).reshape(n,1)
wy_vec = np.array([2,3,4]).reshape(n,1)
x_vec = np.array([[0,1,2],[3,4,5]]).reshape(n,k)
print(x_vec)
regroup_matrix(y_vec, wy_vec, x_vec, n, k)

[[0 1]
 [2 3]
 [4 5]]


array([[1, 2, 0, 1],
       [2, 3, 2, 3],
       [3, 4, 4, 5]])

In [10]:
# "regroup_matrix" test passed

In [11]:
# test get_Znt
n = 3
k = 1
T = 2
x = [np.array([1,2,3]).reshape(n,k),np.array([-1,-2,-3]).reshape(n,k)]
y = [np.array([0,1,2]).reshape(n,1), np.array([0,-1,-2]).reshape(n,1), np.array([0,1,2]).reshape(n,1)]
W_ls = [np.identity(3),np.identity(3),np.identity(3)]

In [12]:
get_Znt(n,k,x,y,W_ls,0)

array([[0., 0., 1.],
       [1., 1., 2.],
       [2., 2., 3.]])

In [13]:
# "get_Znt" test passed

In [14]:
# test get_rnt
n = 3
k = 1
T = 2
x = [np.array([1,2,3]).reshape(n,k),np.array([-1,-2,-3]).reshape(n,k)]
y = [np.array([0,1,2]).reshape(n,1), np.array([0,-1,-2]).reshape(n,1), np.array([0,1,2]).reshape(n,1)]
W_ls = [np.identity(3),np.identity(3),np.identity(3)]
sigma,lam,gamma,rho,beta,t = 1,1,1,1,[1],0

In [15]:
get_rnt(sigma,lam,gamma, rho, beta, t)

array([[-1.],
       [-4.],
       [-7.]])

In [16]:
get_rnt(sigma,lam,gamma, rho, beta, 1)

array([[1.],
       [4.],
       [7.]])

In [17]:
# "get_rnt" test passed

In [18]:
# test get_c  

n = 3
k = 1
T = 2
x = [np.array([1,2,3]).reshape(n,k),np.array([-1,-2,-3]).reshape(n,k)]
y = [np.array([0,1,2]).reshape(n,1), np.array([0,-1,-2]).reshape(n,1), np.array([0,1,2]).reshape(n,1)]
W_ls = [np.identity(3),np.identity(3),np.identity(3)]
sigma,lam,gamma,rho,beta,t = 1,1,1,1,[1],0

In [19]:
get_c(sigma,lam,gamma, rho, beta)

array([[0.],
       [0.],
       [0.]])

In [20]:
# "get_c" test passed.

In [21]:
# test "get_alpha"
n = 3
k = 1
T = 2
x = [np.array([1,2,3]).reshape(n,k),np.array([-1,-2,-3]).reshape(n,k)]
y = [np.array([0,1,2]).reshape(n,1), np.array([0,-1,-2]).reshape(n,1), np.array([0,1,2]).reshape(n,1)]
W_ls = [np.identity(3),np.identity(3),np.identity(3)]
sigma,lam,gamma,rho,beta,t = 1,1,1,1,[1],0

In [22]:
get_alpha_t(sigma,lam,gamma, rho, beta, t)

-4.0

In [23]:
# see modified "get_alpha_t"
def get_alpha_t_mod(sigma,lam,gamma, rho, beta, t):
    l_n = np.ones(n).reshape(n,1)
    rnt = get_rnt(sigma,lam,gamma, rho, beta, t)
    print(rnt)
    c0 = get_c(sigma,lam,gamma, rho, beta)
    print(c0)
    alpha_t = (1/n)*np.matmul(l_n.transpose(),rnt-c0)[0][0]
    return alpha_t

In [24]:
get_alpha_t_mod(sigma,lam,gamma, rho, beta, t)

[[-1.]
 [-4.]
 [-7.]]
[[0.]
 [0.]
 [0.]]


-4.0

In [25]:
# code "get Vnt" then jose

In [26]:
def get_Vnt(n,k,T,x,y,W_ls,params,t):
    # get components
    sigma, lam, gamma, rho = params[:4]
    beta = params[4:]
    ynt = np.array(y[t+1]).reshape(n,1)
    y_lag = np.array(y[t]).reshape(n,1)
    W = np.array(W_ls[t+1]).reshape(n,n)
    W_lag = np.array(W_ls[t]).reshape(n,n)
    xnt = np.array(x[t]).reshape(n,k)
    c0 = get_c(sigma,lam,gamma, rho, beta)
    alpha_t = get_alpha_t(sigma,lam,gamma, rho, beta, t)
    l_n = np.ones(n).reshape(n,1)
    
    # get LHS
    LHS = lam*np.matmul(W,ynt)+gamma*y_lag+rho*np.matmul(W_lag,y_lag)+np.matmul(xnt,np.array(beta).reshape(k,1))+c0+alpha_t*l_n
    
    # get Vnt
    Vnt = np.array(ynt-LHS).reshape(n,1)
    
    return Vnt

In [27]:
# test get_Vnt
n = 3
k = 1
T = 2
x = [np.array([1,2,3]).reshape(n,k),np.array([-1,-2,-3]).reshape(n,k)]
y = [np.array([0,1,2]).reshape(n,1), np.array([0,-1,-2]).reshape(n,1), np.array([0,1,2]).reshape(n,1)]
W_ls = [np.identity(3),np.identity(3),np.identity(3)]
params = [1,1,1,1,1]

In [28]:
get_Vnt(n,k,T,x,y,W_ls,params,t)

array([[ 3.],
       [ 0.],
       [-3.]])

In [29]:
# test passed

In [30]:
# test get_Znt_tilte

def get_Z_tilde(n,k,x,y,W_ls,T,t):
    Y_ls = []
    for q in range(T):
        Y_ls.append(y[q])
    Y_tilde_lag = np.array(y[t]-(1/T)*sum(Y_ls)) # flattened array
        
    WY_ls = []
    for l in range(T):
        W3 = W_ls[l]
        WY_ls.append(np.matmul(W3,np.array(y[l]).reshape(n,1)))
    WY_sum = sum(WY_ls)
    W_lag = W_ls[t]
    WY_tilde_lag = np.matmul(W_lag, np.array(y[t]).reshape(n,1))-(1/T)*(WY_sum)
    # flatten shape
    WY_tilde_lag.ravel()
        
    X_ls = []
    for m in range(T):
        X_ls.append(np.array(x[m]))
    X_sum = sum(X_ls)
    X_tilde = np.array(np.array(x[t])-(1/T)*X_sum).reshape(n,k) 
    
    ravel_X = []
    for p in range(n):
        ravel_X.append(X_tilde[p])
    
    #Z_tilde is n*(k+2)
    Z_tilde = regroup_matrix(Y_tilde_lag,WY_tilde_lag,ravel_X, n, k)
            
    return Z_tilde  

In [31]:
n = 3
k = 1
T = 2
x = [np.array([1,2,3]).reshape(n,k),np.array([-1,-2,-3]).reshape(n,k)]
y = [np.array([0,1,2]).reshape(n,1), np.array([0,-1,-2]).reshape(n,1), np.array([0,1,2]).reshape(n,1)]
W_ls = [np.identity(3),np.identity(3),np.identity(3)]

In [32]:
get_Z_tilde(n,k,x,y,W_ls,T,0)

array([[0., 0., 1.],
       [1., 1., 2.],
       [2., 2., 3.]])

In [33]:
n = 3
k = 2
T = 2
x = [np.array([[1,2,3],[3,2,1]]).reshape(n,k),np.array([[-1,-2,-3],[-3,-2,-1]]).reshape(n,k)]
y = [np.array([0,1,2]).reshape(n,1), np.array([0,-1,-2]).reshape(n,1), np.array([0,1,2]).reshape(n,1)]
W_ls = [np.identity(3),np.identity(3),np.identity(3)]

In [34]:
get_Z_tilde(n,k,x,y,W_ls,T,0)

array([[0., 0., 1., 2.],
       [1., 1., 3., 3.],
       [2., 2., 2., 1.]])

In [35]:
# test passed

In [41]:
# test get_Gnt

def get_Gnt(W_ls, n, lam, t):
    W = W_ls[t]
    S_nt = np.identity(n) - lam*W
    Gnt = np.matmul(W,np.linalg.inv(S_nt)).reshape(n,n)
    return Gnt

In [44]:
n = 3
k = 1
lam = 0.5
W_ls = [np.identity(3),np.identity(3),np.identity(3)]
t = 0

In [45]:
get_Gnt(W_ls, n, lam, t)

array([[2., 0., 0.],
       [0., 2., 0.],
       [0., 0., 2.]])

In [46]:
# test passed

In [110]:
# test get_H_mat

def get_H_mat(x,y,W_ls,lam,sigma,gamma,rho,beta,k,n,T):
    Hnt_vec_ls = []
    c0 = get_c(sigma,lam,gamma, rho, beta)
    for i in range(T):
        #delta is (k+2)*1
        delta = [gamma,rho,*beta]
        #Gnt is n*n
        Gnt = get_Gnt(W_ls, n, lam, i)
        #G_tilde is n*n
        G_ls = []
        for j in range(T):
            G_ls.append(get_Gnt(W_ls, n, lam, j))
        G_tilde = Gnt-(1/T)*sum(G_ls)
    
        #Znt and Z_tilde is n*(k+2)
        Znt = get_Znt(n,k,x,y,W_ls,i)
        Z_tilde = get_Z_tilde(n,k,x,y,W_ls,T,i)

        #GZnt and GZ_tilde is n*(k+2)
        GZnt = np.matmul(Gnt,Znt).reshape(n,k+2)
        GZ_ls = []
        for r in range(T):
            Gnt2 = get_Gnt(W_ls, n, lam, r)
            Znt2 = get_Znt(n,k,x,y,W_ls,r)
            GZ_ls.append(np.matmul(Gnt2,Znt2))
        GZ_tilde = np.matmul(Gnt,Znt)-(1/T)*sum(GZ_ls)
    
        #GZ_delta is n*1
        GZ_delta = np.matmul(GZ_tilde,delta).reshape(n,1)
        GZ_delta_c = GZ_delta - np.matmul(G_tilde, np.array(c0).reshape(n,1))
    
        # reshape H_mat
        ravel_Z_tilde = []
        for p in range(n):
            ravel_Z_tilde.append(Z_tilde[p])
    
        reshape_list = [ravel_Z_tilde,GZ_delta_c.ravel()]
        
        final_reshape_list = []
        for q in range(n):
            row_vec = reshape_list[0][q].tolist()
            row_vec.append(reshape_list[1][q])
            final_reshape_list.append(row_vec)
        
        H_mat = np.array(final_reshape_list).reshape(n,k+3)
        print(H_mat)
        
        #J_n is n*n
        l_n = np.ones(n).reshape(n,1)
        J_n = np.identity(n)-(1/n)*np.matmul(l_n,l_n.transpose())
        
        #Hnt_vec is (k+3)*(k+3)
        Hnt_Jn = np.matmul(H_mat.transpose(),J_n)
        Hnt_vec = np.matmul(Hnt_Jn,H_mat)
        Hnt_vec_ls.append(Hnt_vec)
        
    H = (1/((n-1)*T))*sum(Hnt_vec_ls)
    return H    

In [111]:
n = 3
k = 1
T = 2
x = [np.array([1,2,3]).reshape(n,k),np.array([-1,-2,-3]).reshape(n,k)]
y = [np.array([0,1,2]).reshape(n,1), np.array([0,-1,-2]).reshape(n,1), np.array([0,1,2]).reshape(n,1)]
W_ls = [np.identity(3),np.identity(3),np.identity(3)]
params = [1,0.5,1,1,1]
sigma, lam, gamma, rho = params[:4]
beta = params[4:]

In [112]:
get_H_mat(x,y,W_ls,lam,sigma,gamma,rho,beta,k,n,T)

[[ 0.  0.  1.  2.]
 [ 1.  1.  2.  8.]
 [ 2.  2.  3. 14.]]
[[  0.   0.  -1.  -2.]
 [ -1.  -1.  -2.  -8.]
 [ -2.  -2.  -3. -14.]]


array([[ 1.,  1.,  1.,  6.],
       [ 1.,  1.,  1.,  6.],
       [ 1.,  1.,  1.,  6.],
       [ 6.,  6.,  6., 36.]])

In [113]:
# test passed

In [152]:
def get_first_infoMat(x,y, W_ls,lam,sigma,gamma,rho,beta,k,n,T):
    # (k+4)*(k+4)
    HnT = get_H_mat(x,y, W_ls,lam,sigma,gamma,rho,beta,k,n,T)
    reshape_ls = []
    for i in range(k+3):
        row_vec = HnT[i].tolist()
        row_vec.append(0) # assume that * = 0
        reshape_ls.append(row_vec)
    reshape_ls.append(np.zeros(k+4))
    return np.array(reshape_ls).reshape(k+4,k+4) 

In [153]:
n = 3
k = 1
T = 2
x = [np.array([1,2,3]).reshape(n,k),np.array([-1,-2,-3]).reshape(n,k)]
y = [np.array([0,1,2]).reshape(n,1), np.array([0,-1,-2]).reshape(n,1), np.array([0,1,2]).reshape(n,1)]
W_ls = [np.identity(3),np.identity(3),np.identity(3)]
params = [1,0.5,1,1,1]
sigma, lam, gamma, rho = params[:4]
beta = params[4:]

In [154]:
get_first_infoMat(x,y, W_ls,lam,sigma,gamma,rho,beta,k,n,T)

[[ 0.  0.  1.  2.]
 [ 1.  1.  2.  8.]
 [ 2.  2.  3. 14.]]
[[  0.   0.  -1.  -2.]
 [ -1.  -1.  -2.  -8.]
 [ -2.  -2.  -3. -14.]]


array([[ 1.,  1.,  1.,  6.,  0.],
       [ 1.,  1.,  1.,  6.,  0.],
       [ 1.,  1.,  1.,  6.,  0.],
       [ 6.,  6.,  6., 36.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]])

In [122]:
# test passed

In [141]:
def get_second_infoMat(x,y, W_ls,lam,sigma,gamma,rho,beta,k,n,T):
    # (k+4)*(k+4)
    GJG_JG_tr_ls = []
    JG_tr_ls = []
    for i in range(T):
        Gnt = get_Gnt(W_ls, n, lam, i)
        l_n = np.ones(n).reshape(n,1)
        J_n = np.identity(n)-(1/n)*np.matmul(l_n,l_n.transpose())
        GJ = np.matmul(Gnt.transpose(),J_n).reshape(n,n)
        GJG = np.matmul(GJ,Gnt).reshape(n,n)
        JG = np.matmul(J_n,Gnt).reshape(n,n)
        GJG_tr = np.matrix.trace(GJG)
        JG_tr = np.matrix.trace(JG)
        JG_sqr_tr = np.matrix.trace(np.matmul(JG,JG.transpose()))
        GJG_JG_tr = GJG_tr+JG_tr
        GJG_JG_tr_ls.append(GJG_JG_tr)
        JG_tr_ls.append(JG_tr)
    
    # constant
    first_trace = 1/((n-1)*T)*sum(GJG_JG_tr_ls)
    print(first_trace)
    # constant
    second_trace = 1/(sigma**2*(n-1)*T)*sum(JG_tr_ls)
    print(second_trace)
    
    # putting togther the matrix
    first_k_plus_2_row = []
    for j in range(k+2):
        first_k_plus_2_row.append(np.zeros(k+4))
    
    print(first_k_plus_2_row)
    
    second_row = np.zeros(k+2).tolist()
    print(second_row)
    second_row.append(first_trace)
    print(second_row)
    second_row.append(second_trace)
    print(second_row)
    third_row = np.zeros(k+2).tolist()
    third_row.append(second_trace)
    print(third_row)
    third_row.append(1/(2*sigma**4))
    print(third_row)
    reshape_ls = first_k_plus_2_row
    reshape_ls.extend([second_row,third_row])
    print(reshape_ls)
    
    return np.array(reshape_ls).reshape(k+4,k+4)        


In [146]:
n = 3
k = 1
T = 2
x = [np.array([1,2,3]).reshape(n,k),np.array([-1,-2,-3]).reshape(n,k)]
y = [np.array([0,1,2]).reshape(n,1), np.array([0,-1,-2]).reshape(n,1), np.array([0,1,2]).reshape(n,1)]
W_ls = [np.identity(3),np.identity(3),np.identity(3)]
params = [1,0.5,1,1,1]
sigma, lam, gamma, rho = params[:4]
beta = params[4:]

In [143]:
get_second_infoMat(x,y, W_ls,lam,sigma,gamma,rho,beta,k,n,T)

6.0
2.0
[array([0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0.])]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 6.0]
[0.0, 0.0, 0.0, 6.0, 2.0]
[0.0, 0.0, 0.0, 2.0]
[0.0, 0.0, 0.0, 2.0, 0.5]
[array([0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0.]), [0.0, 0.0, 0.0, 6.0, 2.0], [0.0, 0.0, 0.0, 2.0, 0.5]]


array([[0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 6. , 2. ],
       [0. , 0. , 0. , 2. , 0.5]])

In [144]:
# test passed

In [148]:
def get_info_mat(x,y, W_ls,lam,sigma,gamma,rho,beta,k,n,T):
    # this is a (k+4)*(k+4) matrix
    first_info_mat = get_first_infoMat(x,y, W_ls,lam,sigma,gamma,rho,beta,k,n,T)
    second_info_mat = get_second_infoMat(x,y, W_ls,lam,sigma,gamma,rho,beta,k,n,T)
    info_mat = 1/(sigma**2)*first_info_mat+second_info_mat
    info_mat.reshape(k+4,k+4)
    return info_mat

In [149]:
n = 3
k = 1
T = 2
x = [np.array([1,2,3]).reshape(n,k),np.array([-1,-2,-3]).reshape(n,k)]
y = [np.array([0,1,2]).reshape(n,1), np.array([0,-1,-2]).reshape(n,1), np.array([0,1,2]).reshape(n,1)]
W_ls = [np.identity(3),np.identity(3),np.identity(3)]
params = [1,0.5,1,1,1]
sigma, lam, gamma, rho = params[:4]
beta = params[4:]

In [150]:
get_info_mat(x,y, W_ls,lam,sigma,gamma,rho,beta,k,n,T)

[[ 0.  0.  1.  2.]
 [ 1.  1.  2.  8.]
 [ 2.  2.  3. 14.]]
[[  0.   0.  -1.  -2.]
 [ -1.  -1.  -2.  -8.]
 [ -2.  -2.  -3. -14.]]


array([[ 1. ,  1. ,  1. ,  6. ,  0. ],
       [ 1. ,  1. ,  1. ,  6. ,  0. ],
       [ 1. ,  1. ,  1. ,  6. ,  0. ],
       [ 6. ,  6. ,  6. , 42. ,  2. ],
       [ 1. ,  1. ,  1. ,  3. ,  1.5]])

In [151]:
# test passed 